__DISTRIBUTION STATEMENT B:__ Distribution authorized to U.S. Government Agencies Only: Proprietary Information; _DFARS SBIR/STTR data rights - DFARS 252.227-7018_ 

__LIMITED RIGHTS: Contract No.: N68936-20-C-0054; Contractor Name: 0 Base Design LLC; Contractor Address: 5107 Unicon Dr, Unit K, Wake Forest NC 27587-5020;__ The Government's rights to use, modify, reproduce, release, perform, display, or disclose these technical data are restricted by paragraph (b)(2) of the Rights in Noncommercial Technical Data and Computer Software - Small Business Innovation Research (SBIR) Program clause contained in the above identified contract. Any reproduction of technical data or portions thereof marked with this legend must also reproduce the markings. Any person, other than the Government, who has been provided access to such data must promptly notify the above named Contractor.

In [1]:
# pull packages
import numpy as np
import scipy.optimize as spopt
import scipy.fftpack as spfft
import scipy.ndimage as spimg
import cvxpy as cvx
import pretty_plotly as pp

import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.set_cmap('viridis_r')
%matplotlib inline

In [72]:
def snr_to_amp(snr):
    '''
    Convert SNR (dB power) to signal amplitude, relative to unity.
    
    Parameters
    ----------
    snr : float
    
    Example
    -------
    snr_to_amp(3)
    >>> 1.9952623149688795
        
    '''
    return np.power(10, snr/10)

def amp_to_snr(amp):
    '''
    Convert signal amplitude to dB, relative to unity.
    
    Parameters
    ----------
    amp : float
    
    Example
    -------
    amp_to_snr(100)
    >>> 20

    '''    
    return 10*np.log10(amp)
    

def subsample(y, t, ndct, fsADC, fsCS, channels):
    '''
    Generate a signal samples at fs, that is end-start seconds 
    long consisting of noise and tones. tones is a zipped list 
    of frequencies relative to fs and amplitudes in snr relative
    to the noise floor. 
    
    Parameters
    ----------
    y : array, signal
    t : array, time in seconds
    ndct : int, frequency resolution
    fsADC : int, sampling frequency of ADC
    fsCS : int, mininum time resolution of spatial samples
    channels : int, number of spatial samples, number of ADC
        
    '''   

    numReducedSamples = np.int(fsCS/fsADC)
    
    # to avoid large matrix mulitplies we limit the maximum legnth 
    # of the signal to ndct. The Vandamonde matrix is square
    # before we subsample, however we only subsample the rows
    # Thus a long time signal will require more memory
    # 
    ri = np.arange(0, ndct, numReducedSamples, int)
    position = [ri]

    for ii in [1, 2, 3, 4, 5, 6, 7]:
        position.append(ri+ii), 
    ri = np.array(position).flatten()
    ri.sort() # sorting not strictly necessary, but convenient for plotting
    ri = ri[ri<ndct]
    
    t2 = t[ri]
    y2 = y[ri]    
    return y2, t2, ri


def l1_convex_optimization(A, y2, eps=1e-2, verbose=True):
    '''
    Perform l1 constrained optimization. A is the Vandermonde
    matrix (dft kernel), y2 is the measured signal.
    
    Parameters
    ----------
    A : array, transform matrix, nxm
    y2 : array, measurement vecotr, nx1
    eps : float, error in constraint
    verbose : boolean
        
    '''   
    vx = cvx.Variable(A.shape[1]) 
    objective = cvx.Minimize(cvx.norm(vx, 1))
    constraints = [cvx.abs(A*vx - y2) <= eps]
    prob = cvx.Problem(objective, constraints)
    result = prob.solve(verbose=verbose)
    return vx, result

def complex_tones_with_noise(fs, tones, start=0, end=1):
    '''
    Generate a signal samples at fs, that is end-start seconds 
    long consisting of noise and tones. tones is a zipped list 
    of frequencies relative to fs and amplitudes in snr relative
    to the noise floor. 
    
    Parameters
    ----------
    fs : float
    tones : generator of (frequency, snr) tuples
    start : float start time in seconds
    end : float end time in seconds
        
    '''    
    numSamples = np.floor((end-start)*fs).astype(int)
    t = np.linspace(start, end, numSamples)
    y = np.array([0])
    count = 0
    for f,snr in tones:
        y = y + snr_to_amp(snr)*np.exp(2j*np.pi*f * t)
        count = count + 1
    y = y + (np.random.randn(len(y))+np.random.randn(len(y))*1j)  
    return y,t,count

def cognitive_imag(y2, ri, fs, NFFT, verbose=False):
    '''
    Cognitive Spectral Esitmation of complex spectra, imaginary 
    of a set of tones with varying SNR and frequency.
    
    Parameters
    ----------
    tones : generator of (frequency, snr) tuples
    fs : float, sampling frequency
    NFFT : int, frequency resolution
    end : float end time in seconds
    samplesPerTone: int, number of samples
    numChannels: int, number of spatial channels (number of ADC)
    verbose: booolean
    '''    

    # create idct matrix operator
    A = spfft.idst(np.identity(NFFT), axis=0)
    A = A[ri] # choose rows
        
    vx,_ = l1_convex_optimization(A, y2, verbose=verbose)
    return vx
    
    
def cognitive_real(y2, ri, fs, NFFT, verbose=False):
    '''
    Cognitive Spectral Esitmation of complex spectra, real 
    of a set of tones with varying SNR and frequency.
    
    Parameters
    ----------
    tones : generator of (frequency, snr) tuples
    fs : float, sampling frequency
    NFFT : int, frequency resolution
    end : float end time in seconds
    samplesPerTone: int, number of samples
    numChannels: int, number of spatial channels (number of ADC)
    verbose: booolean
    '''    
   
    # create idct matrix operator
    A = spfft.idct(np.identity(NFFT), axis=0)
    A = A[ri] # choose rows
    
    vx,_ = l1_convex_optimization(A, y2, verbose=verbose)
    return vx
        
    
def run_simulation(tones, fs, sr, NFFT, numChannels=8, verbose=False):
    fc = fs*sr
    y,t,numTones = complex_tones_with_noise(fc, tones, end=NFFT/fc)
    y2,t2,ri = subsample(y, t, NFFT, fs, fc, numChannels)
    real_output = cognitive_real(np.real(y2), ri, fc, NFFT)
    imag_output = cognitive_imag(np.imag(y2), ri, fc, NFFT)
    return [real_output, imag_output, y, t, ri]
 
    
def plot_spectra(data, fs, sr): 
    fc = fs*sr
    spec_i = np.array(data[0].value).squeeze()
    spec_q = np.array(data[1].value).squeeze()
    spec = np.sqrt((spec_i**2) + (spec_q**2))
    sig = spfft.dct(spec, axis=0)
#     psd = 10*np.log10(np.abs(spec))
#     pp.line_plotly(psd, n=['psd'], x=[np.arange(len(spec))/len(frespecqSig)*fs/2], xTitle= 'frequency  [Hz]', yTitle= 'PSD [dB]')
    pp.line_plotly(np.abs(spec), n=['psd'], x=[np.arange(len(spec))/len(spec)*fc/2], xTitle= 'frequency  [Hz]', 
                   yTitle= 'PSD', title='Cognitive Magnitude Spectra')
    
def plot_sig(data,num_chan):
    _,_,y,t,ri = data
    i = np.real(y)
    q = np.imag(y)
    ic = i[ri]
    qc = q[ri]
    nyq = ri[::num_chan+1]
    iny = i[nyq]
    qny = q[nyq]
    pp.line_scatter_plotly([i,q,ic,qc,iny,qny], [np.arange(len(i)), np.arange(len(q)), ri, ri, nyq, nyq], 
                          xTitle= 'sample count', yTitle='amplitude', title='sampled signal');
    

## __COGNITIVE IQ SAMPLING AND COMPLEX SPECTRAL RECONSTRUCTION__

Cognitive Sampling is a sub-Nyquist sampling methodology that can be implemented in commercial of the shelf (COTS) hardware. A block diagram of this cognitive sampling system is shown below. Intuitively the cognitive sampling system samples in time and spatially over a bank of analog to digital converters (ADCs). This enables us to reformulate spectral reconstruction as an L1 - optimization problem. We are essentially harnessing computational power to reduce sampling constraints and instantaneously interrogate large bandwidths. 

![Cognitive Sampling Block Diagram](block.png)

In the exposition that follows we examine several cognitive sampling scenarios to illustrate how this subsystem functions and to delineate the difference between this sampling method and Nyquist sampling.  Each simulation is capable of instantaneously interrogating 500 MHz of bandwidth. In each example the sampling rate of the ADC is set to 10 MHz, the spatial sampling time is set to 0.83 nsec, and we have a bank of 8 ADC. In each case we cognitively sample 15.3 microseconds worth of data, approximately 1500 sample, to recover the complex spectra.

In [94]:
SAMPLING_FREQENCY = 1e6
SPATIAL_RESOLUTION = 1e3
FREQUENCY_RESOLUTION = 500*10
SPATIAL_CHANNELS = 8
def test_tones(tones):
    output = run_simulation(tones, fs=SAMPLING_FREQENCY, sr=SPATIAL_RESOLUTION, 
                            NFFT=FREQUENCY_RESOLUTION, numChannels=SPATIAL_CHANNELS)
    plot_spectra(output, SAMPLING_FREQENCY, SPATIAL_RESOLUTION)
    #plot_sig(output, SPATIAL_CHANNELS)  
    
tones = zip([203e6],[50])
test_tones(tones)    

## __EXAMPLE 1: Low Frequency Tone__

We simulate a complex 2 MHz tone with a signal to noise ratio (SNR) of 10 dB.  This is a trivial case as we are well below the Nyquist criteria for a single ADC. However, it clearly illustrates the difference in sampling schemes. The bank of eight ADC depicted in the block diagram correspond to the cognitive samples in the time domain figure. The Nyquist and each of the bank of 8 ADC sample at 10 MHz. It is clear that an input signal at this frequency should not have any issues being resolved by either sampling method. We depict here that we are cognitively sampling IQ data and reconstructing the spectra and phase charasteritcs of the signal. 


In [82]:
tones = zip([10e6],[10])
test_tones(tones)

## __EXAMPLE 2: Tone with Frequency Greater than Nyquist__

We simulate a complex 350 MHz tone with a signal to noise ratio (SNR) of 10 dB.  In the case of Nyquist sampling we would have a great deal of aliasing. This can be seen in the time domain plot. We illustrate here that with cognitive spectral reconstruction we can reproduce the phase and the spectra of the given signal. To reiterate this is accomplished by analyzing 205 microseconds of the input data, approximately 2050 samples. This reduces data bandwidth requirements in processing and enables analysis of large instantaneous bandwidths, without channelization and down conversion. 


In [139]:
tones = zip([350e6],[10])
output = run_simulation(tones, fs=SAMPLING_FREQENCY, sr=SPATIAL_RESOLUTION, 
                        NFFT=FREQUENCY_RESOLUTION, numChannels=512*4)
plot_spectra(output, SAMPLING_FREQENCY, SPATIAL_RESOLUTION)
plot_sig(output, SPATIAL_CHANNELS)

## __EXAMPLE 3: Multiple Tones __

We simulate two complex tones at 302 MHz and 305 MHz  with a signal to noise ratio (SNR) of 10 dB.  Both tones are above the Nyquist sampling criteria and result in an aliased time domain signal. Through cognitive spectral reconstruction we are able to resolve these two signals. We note here that we increased the frequency resolution, which increases the duration of the signal under test to 205 microsecond.


In [140]:
tones = zip([302e6, 305e6],[10, 10])
output = run_simulation(tones, fs=SAMPLING_FREQENCY, sr=SPATIAL_RESOLUTION, NFFT=512*4, numChannels=SPATIAL_CHANNELS)
plot_spectra(output, SAMPLING_FREQENCY, SPATIAL_RESOLUTION)
plot_sig(output, SPATIAL_CHANNELS)

## __EXAMPLE 4: Simulated Emitter __

We reconstruct the Complex RF Spectra of a simulated emitter. We use our simulation software to create a linear frequency modulated emitter with a carrier frequency of 350 MHz and 70 MHz pulse bandwidth. For this simulation we set the sample rate of the ADC to 100 MHz and a spatial resolution of 0.83 nanoseconds. 

In [ ]:
from pyvalcore.utils import read_from_pickle

SAMPLING_FREQENCY = 100e6
SPATIAL_RESOLUTION = 120
FREQUENCY_RESOLUTION = 512*4
SPATIAL_CHANNELS = 8

fs = SAMPLING_FREQENCY
sr = SPATIAL_RESOLUTION
x = read_from_pickle('radar_lfm_example.pickle')
sig = x['radar_samples']
time = x['radar_time']
NFFT = 512
fc = fs*sr
y2,t2,ri = subsample(sig, time, FREQUENCY_RESOLUTION, fs, fc, SPATIAL_CHANNELS)
real_output = cognitive_real(np.real(y2), ri, fc, FREQUENCY_RESOLUTION)
imag_output = cognitive_imag(np.imag(y2), ri, fc, FREQUENCY_RESOLUTION)
plot_spectra([real_output, imag_output], SAMPLING_FREQENCY, SPATIAL_RESOLUTION)